In [4]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [5]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm_notebook(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [99]:
# # Scraping most recent reviews
# url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
# df_recent = ratebeer_scraper(0,100,url)

# # Scraping new beers
# url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
# df_new = ratebeer_scraper(0,100,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,100,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,100,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,100,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,100,url)

/Users/lukebetham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [100]:
df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [101]:
df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.to_csv('all_reviews.csv',index=False)


18669
10530


In [102]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
3,Oskar Blues Death By King Cake,1.6,-1.6,Oskar Blues Brewery (CANarchy Craft Brewery Co...,"Can. Not a porter, has nothing to do with dark...",cheap
4,Glaabsbräu Hefeweizen Alkoholfrei,3.7,+1.6,Privatbrauerei Glaab,28-04-20 // 500ml bottle. NEW RECIPE! Really n...,rhoihessegold
5,Pearl Street Shitty Lyte Beer,4.3,+1.6,Pearl Street Brewery,Hints of grassy aroma. Taste has a grainy cere...,seuinoz
6,Red Brick HopLanta,1.7,-1.5,Atlanta Brewing Company,"Birra torbida, schiuma bianca grossolana e abb...",pich.83


In [103]:
df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')

## Get more information on all of the beers - Selenium

In [119]:
from selenium import webdriver
import time

In [169]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver')

for beer in tqdm_notebook(df.beer_name.unique()[185:]):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

/Users/lukebetham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [168]:
# first_184 = desc_string

In [176]:
beer_deets = pd.DataFrame(first_184+desc_string, columns = ['return_string'])

In [177]:
beer_deets["name_searched"] = df.beer_name.unique()

In [178]:
beer_deets

,return_string,name_searched
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,Shepherd Neame Christmas Ale
...,...,...
6260,Lamplighter / Cambridge Overnight Oats\n🇺🇸 Sou...,Lamplighter / Cambridge Overnight Oats
6261,Tired Hands / Omnipollo Double Milkshake IPA -...,Tired Hands / Omnipollo Double Milkshake IPA -...
6262,Black Iris Mother Knows Best\n🇬🇧 IPA • 6.2%\n-...,Black Iris Mother Knows Best
6263,Great Lakes HazeMama\n🇨🇦 IPA - Hazy / NEIPA • ...,Great Lakes HazeMama


In [179]:
beer_deets.to_csv('beer_deets.csv',index=False)